In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
from google.colab import drive, files
drive.mount('/content/drive')
rootpath = '/content/drive/Shareddrives/KW phase1 repo'

Mounted at /content/drive


In [3]:
import time
def current_milli_time():
    return round(time.time() * 1000)

In [4]:
import sys
sys.path.append(rootpath+'/package')
from utils.env_sdp import env
import pickle
import numpy as np
import csv

In [5]:
from model.uav import UAV, setUAVEnv
from model.sdpv3 import calculateSDP, setSDPEnv
from model.pathlossmodel import PathLossModel, NoiseMaker, setPathLossModelEnv

from service.distributeuavs import createUAVsSamples,setDistributeUavsEnv , TestEnv
from service.measurerss import measureRSS, setMeasuerRssEnv
from service.combinationset import combinationSet , setCombinationSetEnv
from multiprocessing import Pool

setDistributeUavsEnv("sdp")
TestEnv()
setCombinationSetEnv("sdp")
setPathLossModelEnv("sdp")
setMeasuerRssEnv("sdp")
setUAVEnv("sdp")
setSDPEnv("sdp")

envName sdp
16


In [6]:
savePath = '/content/drive/Shareddrives/KW phase1 repo/datas/anomalySDPResult/0808_3_v2'
rootPath = '/content/drive/Shareddrives/KW phase1 repo'
!mkdir /content/drive/Shareddrives/KW\ phase1\ repo/datas/anomalySDPResult/0808_3_v2

mkdir: cannot create directory ‘/content/drive/Shareddrives/KW phase1 repo/datas/anomalySDPResult/0808_3_v2’: File exists


In [7]:
deviations = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [8]:
GetDevicationStr = lambda deviation: str(int(deviation*100))

In [ ]:
from model.uav import UAV
MAXPICKERROR = 3
for deviation in deviations:
  for ipickError in np.arange(0,MAXPICKERROR+1,1):
    import pickle as dill
    file = open(savePath+"/anomalyUAVs_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_300.npy","wb")
    file.close()
    file = open(savePath+"/UAVsSamples_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_300.npy","wb")
    file.close()
    for j in range(env.N_ngps):
      file = open(savePath+"/test_position{}_pickerror{}_deviation{}_300.csv".format(j,ipickError,GetDevicationStr(deviation)),"wb")
      file.close()

def RunSDP(deviation,ipickError,UAVsSamples):
  result = {}
  for iSample, UAVsSample in UAVsSamples.items():
    _pathLossModel = PathLossModel()
    _pathLossModel.noiseMaker = NoiseMaker(mean = env.noiseMean, deviation = deviation) 
    print(iSample)
    result[iSample] = calculateSDP(combinationSet({0:UAVsSample}), {0:UAVsSample}, _pathLossModel)[0]
    # print(result[iSample])
    for j in range(env.N_ngps):
      with open(savePath+"/test_position{}_pickerror{}_deviation{}_300.csv".format(j,ipickError,GetDevicationStr(deviation)), "a") as _csv:
        csvWriter = csv.writer(_csv, delimiter=',')
        csvWriter.writerows(result[iSample][j])
    print()
    anomalyUAVs = []
    for iUAV, UAV in UAVsSample.items():
      if UAV.observedPosition != None and UAV.observedPosition != UAV.realPosition:
        # print(UAV.observedPosition)
        # print(UAV.realPosition)
        anomalyUAVs.append(UAV.id)
    with open(savePath+"/anomalyUAVs_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_300.npy", 'ab') as handle:
      dill.dump(anomalyUAVs, handle)
    with open(savePath+"/UAVsSamples_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_300.npy", 'ab') as handle:
      dill.dump(UAVsSample, handle)

from multiprocessing import Manager, Process
for deviation in deviations:
  for ipickError in np.arange(0,MAXPICKERROR+1,1):
    _pathLossModel = PathLossModel()
    _pathLossModel.noiseMaker = NoiseMaker(mean = env.noiseMean, deviation = deviation) 
    with open(rootPath+"/datas/uavPosition/300_0706/done_deviation/"+GetDevicationStr(deviation)+"/UAVsSamples.pkl", "rb") as f:
      UAVsSamples = pickle.load(f)
    # print(len(UAVsSamples[0]))
    # UAVsSamples = {0:UAVsSamples[0]}
    for iSample, UAVSample in UAVsSamples.items():
      for pickTime in range(ipickError):
        print("pickTime",pickTime)
        uavList_gps = []
        for id, uav in UAVSample.items():
          if not uav.observedPosition == None and uav.observedPosition == uav.realPosition:
            uavList_gps.append(id)
        pickedAnomalyIndex = np.random.choice(uavList_gps, 1, replace=False)[0]
        # print("pickedAnomalyIndex",pickedAnomalyIndex)
        originObservedPosition = UAVSample[pickedAnomalyIndex].observedPosition
        shift = np.random.choice(np.arange(400, 800), 3, replace=False)
        anomalyObservedPosition = [originObservedPosition[0]+shift[0], originObservedPosition[1]+shift[1], originObservedPosition[2]+shift[2]]
        UAVSample[pickedAnomalyIndex].observedPosition = anomalyObservedPosition
    # for iSample, UAVsSample in UAVsSamples.items():
    #   for iUAV, UAV in UAVsSample.items():
    #     if UAV.observedPosition != None:
    #       print(iSample,iUAV,UAV.observedPosition)
    #       print(iSample,iUAV,UAV.realPosition)
    measureRSS(UAVsSamples, _pathLossModel)  
    
    print("sub")
    p = Process(target=RunSDP, args=(deviation,ipickError,UAVsSamples))
    p.Daemon = True
    p.start()
for deviation in deviations:
  for ipickError in np.arange(0,MAXPICKERROR+1,1):
    p.join()

串流輸出內容已截斷至最後 5000 行。
██████████████████████████████████████████████████████████████████████

39
██████████████████████████████████████████████████████████████████████████████████████████

39
████████████████████████
█
39
█████████████████████████████████████████████████
██
39
█████████████████████████████

█39
██████████████████████████
██
39
████████████████████████████████████████████████████████████████████████
█
39
██████████████████████████████████

39
███████████████████████████████████

39
█
█
39
███

39
████████████████████

█39
███
██
█39
█████████████

39
██████
█
39
████████████████████████████████████

█39
███████████
███
39
███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

39
█████████████████

██39
███████████████████████████████████████████████
██
39
████████████████████████████████████████████████████████████

39
█████████
█
█39
██████████████████

39
█████

39
███████████████

In [ ]:
from multiprocessing import Manager
optimalMSE = {}
optimalMSEGroup = {}
manager = Manager() 
qOptimalMSE = manager.dict()
qOptimalMSEGroup = manager.dict()
def runVAEOutput(iSample,_qOptimalMSE,_qOptimalMSEGroup):
  print("iSample",iSample)
  _optimalMSE = []
  _optimalMSEGroup = []
  UAVsSample = UAVsSamples[iSample]
  
  for iState in allEstimateGroups[iSample]:
    try:
      mse = MSE(vaeOutputs[iSample][iState], normalizeVAETestingDatas[iSample][iState])
      if iState %10 == 0:
        print("iState:",iSample,iState)
      _optimalMSE.append(mse)
      _optimalMSEGroup.append(states[iSample][iState])
    except:
      continue    
  _qOptimalMSE[iSample] = _optimalMSE
  _qOptimalMSEGroup[iSample] = _optimalMSEGroup
  print("finish")
from multiprocessing import Process, Queue
for iSample in UAVsSamples:
  print("sub")
  p = Process(target=runVAEOutput, args=(iSample,qOptimalMSE,qOptimalMSEGroup,))
  p.Daemon = True
  p.start()
for iSample in UAVsSamples:
  p.join()
optimalMSE = dict(qOptimalMSE)
optimalMSEGroup = dict(qOptimalMSEGroup)

In [ ]:
import pickle as dill
anomalyUAVs = []
logPick = 0
logDiv = 2
with open(rootpath+"/datas/anomalySDPResult"+"/0804/anomalyUAVs_pick{}div{}_300.npy".format(logPick,logDiv), 'rb') as f:
  while True:
    try:
      anomalyUAVs.append(dill.load(f))
    except EOFError:
      break
print(anomalyUAVs)

In [ ]:
import pickle as dill
UAVsSamples = {}
logPick = 0
logDiv = 2
with open(rootpath+"/datas/anomalySDPResult"+"/0804/UAVsSamples_pick{}div{}_300.npy".format(logPick,logDiv), 'rb') as f:
  counter = 0
  while True:
    try:
      UAVsSamples[counter] = dill.load(f)
      counter += 1
    except EOFError:
      break
print(UAVsSamples)
import csv
sdpResultsDatas = []
for j in range(env.N_ngps):
  with open(rootpath+"/datas/anomalySDPResult/0804/test_position{}_pickerror{}_deviation{}_300.csv".format(j,logPick,logDiv), "r") as _csv:
    csvReader = list (csv.reader(_csv, delimiter=',', quoting=csv.QUOTE_NONNUMERIC))
    sdpResultsDatas.append(csvReader)

In [ ]:
np.array(sdpResultsDatas).shape

In [ ]:
sdpResultsDatas

In [ ]:
sdpResultsDatas = np.array(sdpResultsDatas).reshape(4,1,7,3)

In [ ]:
# with open(rootPath+"/datas/uavPosition/300_0706/deviation/"+GetDevicationStr(deviation)+"/UAVsSamples.pkl", "rb") as f:
#   UAVsSamples = pickle.load(f)
# for uavSample, uav in UAVsSamples[0].items():
#   print(uav.realPosition)

In [ ]:
transposeData = sdpResultsDatas.transpose((1,2,0,3))

In [ ]:
np.array(transposeData).shape

In [ ]:
transposeData

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,12))
ax = plt.axes(projection='3d')
ax.set_xlabel('x-axis (m)', fontsize=18)
ax.set_ylabel('y-axis (m)', fontsize=18)
ax.set_zlabel('heights (m)', fontsize=18)
plt.grid()
for isample, groups in enumerate(transposeData):
  for isample,uavsPostitions in enumerate(groups):
    for position in uavsPostitions:
      x = position[0]
      y = position[1]
      z = position[2]
      scatter3 = ax.scatter(x, y, z, c='g', marker='o')
# for iUAV, PostionsOfGroupOfSample in enumerate(sdpResultsDatas):
#   for iSample, PostionsOfGroup in enumerate(PostionsOfGroupOfSample):
#     for iPostions, position in enumerate(PostionsOfGroup):
#       x = position[0]
#       y = position[1]
#       z = position[2]
#       scatter3 = ax.scatter(x, y, z, c='g', marker='o')
for iSample, UAVsSample in UAVsSamples.items():
  for iUAV, uav in UAVsSample.items():
    if uav.observedPosition == None:
      x = uav.realPosition[0]
      y = uav.realPosition[1]
      z = uav.realPosition[2]
      scatter3 = ax.scatter(x, y, z, c='r',s=50, marker='x')
  break

In [ ]:
import time
timeStr = str(round(time.time() * 1000))